# **Quick Plots**

## *Setup*

### Imports

Minimal imports for plotting and what is needed to extract variables.

In [1]:
from wrf import getvar, to_np, latlon_coords, get_cartopy
from netCDF4 import Dataset #type: ignore
import numpy as np
import cartopy.crs as crs
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
from matplotlib.cm import get_cmap

### Variable List and Read-in

Sets the list of variables to plot and reads-in the WRF file.

In [ ]:
vari = ['EBIO_RES','EBIO_GEE','E_CO2','CO2_BIO','CO2_BCK','CO2_ANT']

wrf_f = Dataset('./wrfout_d02_2023-07-21_18:00:00')

## *Plotting*

### Main Plot Loop

Loops through the variable list to plot each variable. Converts flux variables from mol km<sup>-2</sup> hr<sup>-1</sup> to gC m<sup>-2</sup> day<sup>-1</sup>

In [14]:
golden = (1. + np.sqrt(5.))/2.
figsize = (12., 12./golden)

for var in vari:
    if var not in ['EBIO_RES','EBIO_GEE']:
        v = getvar(wrf_f, var)[0,...]
        #if var == 'E_CO2':
         #   v = v * 0.00029
    else:
        v = getvar(wrf_f, var)# * 0.00029 #mol km^-2 hr^-1 -> gC m^-2 day^-1
    lats, lons = latlon_coords(v)
    cart_proj = get_cartopy(v)
    if var in ['EBIO_RES','EBIO_GEE','E_CO2']:
        v = v * 0.0029
    min_v = np.min(v.data)
    max_v = np.max(v.data)
    print(f'{var}: {min_v = }, {max_v = }')
    if var in ['EBIO_RES','EBIO_GEE','E_CO2']:
        levels = np.linspace(min_v, max_v, num=12)
        cbar_levels = np.linspace(min_v,max_v,num=12)
    else:
        levels = np.linspace(min_v, max_v, num=10)
        cbar_levels = np.linspace(min_v,max_v,num=10)
    fig, ax = plt.subplots(1,1, subplot_kw={'projection': cart_proj},figsize=figsize)
    states = cfeature.STATES
    ax.add_feature(states, linewidth=0.5, edgecolor='black')
    contours = ax.contourf(to_np(lons), to_np(lats), to_np(v), levels=levels, cmap=plt.get_cmap('rainbow'), transform=crs.PlateCarree())
    fig.colorbar(contours, ax=ax, orientation='horizontal', pad=0.05, ticks=cbar_levels, fraction=0.052, label=f'{var} {"gC m^-2 day^-1" if var in ["EBIO_RES","EBIO_GEE","E_CO2"] else "ppmv"}')
    ax.set_title(f'{var} at 2023-07-21 18:00Z')
    plt.savefig(f'{var}_2023-07-21_18z.png')
    plt.close()

EBIO_RES: min_v = 0.0, max_v = 41.24482
EBIO_GEE: min_v = -74.94712, max_v = 0.0
E_CO2: min_v = 0.0, max_v = 26352.715
CO2_BIO: min_v = 403.2116, max_v = 411.2159
CO2_BCK: min_v = 402.92795, max_v = 410.4399
CO2_ANT: min_v = 403.16763, max_v = 516.4101
